In [1]:
import re
import numpy as np
import constraint
with open("data.txt") as file:
    tiles = file.read().split("\n\n")

In [2]:
class Tile:
    
    def __init__(self, id_, tile):
        self.id_ = id_
        self.tile = tile
        
    def get_all_possible_borders(self):
        
        top = self.tile[0, :]
        right = self.tile[:, -1]
        left = self.tile[:, 0]
        bottom = self.tile[-1, :]
        
        top_rev = self.tile[0, :][::-1]
        right_rev = self.tile[:, -1][::-1]
        left_rev = self.tile[:, 0][::-1]
        bottom_rev = self.tile[-1, :][::-1]
        
        return set(list(map(tuple, [top, right, left, bottom, top_rev, right_rev, left_rev, bottom_rev])))
    
    def transform(self, state):
        rotation, hori, vert = state
        result = np.rot90(self.tile, k=rotation)
        if hori: 
            result = np.fliplr(result)
        if vert: 
            result = np.flipud(result)
            
        return Tile(self.id_, result)
    
    def aligns(self, other_tile, on):
        """
        On: this tile's relative position to the other tile: top, right, bottom, left
        """
        if on == "top":
            this_border = self.tile[-1, :] # Bottom 
            other_border = other_tile.tile[0, :] # Top
        elif on == "right":
            this_border = self.tile[:, 0] # Left
            other_border = other_tile.tile[:, -1] # Right
        elif on == "bottom":
            this_border = self.tile[0, :] # Top
            other_border = other_tile.tile[-1, :] # Bottom
        else: # on == "left"
            this_border = self.tile[:, -1] # Right
            other_border = other_tile.tile[:, 0] # Left
        
        return all(this_border == other_border)
    
    def remove_borders(self):
        return self.tile[1:-1, 1:-1]
        
    def __repr__(self):
        return str(self.tile)

In [3]:
id_to_tile = {}
for tile in tiles:
    inputs = tile.strip().split("\n")
    id_ = int(re.findall("\d+", inputs[0])[0])
    id_to_tile[id_] = Tile(id_, np.array(list(map(list, inputs[1:]))))

In [4]:
id_to_shares = {}
for id_ in id_to_tile:
    shares = set()
    curr_border = id_to_tile[id_].get_all_possible_borders()
    
    for id_other in id_to_tile:
        if id_ != id_other:
            other_border = id_to_tile[id_other].get_all_possible_borders()
            if len(other_border.intersection(curr_border)) != 0:
                shares = shares.union([id_other])
    
    id_to_shares[id_] = shares

In [5]:
total = 1
# Just check for tiles that only share 2 borders. These must be corners.
for id_ in id_to_shares:
    if len(id_to_shares[id_]) == 2:
        total *= id_

In [6]:
total

23497974998093

In [7]:
# Part 2
import itertools

In [8]:
len_to_edges = {2: [], 3: [], 4: []}
for id_ in id_to_shares:
    len_to_edges[len(id_to_shares[id_])].append(id_)

In [9]:
locationProblem = constraint.Problem()
x_max, y_max = 11, 11

for position in itertools.product(range(12), repeat=2):
    x, y = position
    if (x, y) in itertools.product([0, 11], repeat=2):
        locationProblem.addVariable(position, len_to_edges[2])
    elif x == 0 or x == x_max or y == 0 or y == y_max:
        locationProblem.addVariable(position, len_to_edges[3])
    else:
        locationProblem.addVariable(position, len_to_edges[4])

In [10]:
def can_connect(tile_1, tile_2):
    return tile_2 in id_to_shares[tile_1]

In [11]:
def get_connections(position):
    x, y = position
    neighbors = []
    if x != 0:
        neighbors.append((x-1, y))
    if x != x_max:
        neighbors.append((x+1, y))
    if y != 0:
        neighbors.append((x, y-1))
    if y != y_max:
        neighbors.append((x, y+1))
    
    return neighbors
    

In [12]:
for position in itertools.product(range(12), repeat=2):
    for connection in get_connections(position):
        locationProblem.addConstraint(can_connect, [connection, position])
        
locationProblem.addConstraint(constraint.AllDifferentConstraint())

In [13]:
pos_to_id = locationProblem.getSolution()
id_to_pos = {id_: pos for pos, id_ in pos_to_id.items()}

In [14]:
# set up another CSP with the configurations for each tile

In [15]:
configProblem = constraint.Problem()
for id_ in id_to_pos:
    possible_configs = list(itertools.product([id_], [0, 1, 2, 3], [False, True], [False, True]))
    configProblem.addVariable(id_, possible_configs)

In [16]:
def get_relative(tile_1, tile_2):
    """
    tile_1, tile_2 : their IDs
    returns relative position of tile_1 to tile_2
    """
    tile_1_pos = id_to_pos[tile_1]
    tile_2_pos = id_to_pos[tile_2]
    
    if tile_1_pos[0] < tile_2_pos[0]: 
        return "left"
    elif tile_1_pos[0] > tile_2_pos[0]:
        return "right"
    elif tile_1_pos[1] < tile_2_pos[1]:
        return "top"
    else: # tile_1_pos[1] > tile_2_pos[2]:
        return "bottom"
    
def check_aligns(config_1, config_2):
    tile_1, *tile_1_config = config_1
    tile_2, *tile_2_config = config_2
    on = get_relative(tile_1, tile_2)
    
    tile_1 = id_to_tile[tile_1]
    tile_2 = id_to_tile[tile_2]
    return tile_1.transform(tile_1_config).aligns(tile_2.transform(tile_2_config), on=on)
    

In [17]:
for id_ in id_to_pos:
    curr_pos = id_to_pos[id_]
    for connection in get_connections(curr_pos):
        other_id = pos_to_id[connection]
        configProblem.addConstraint(check_aligns, [id_, other_id])

In [18]:
id_to_config = configProblem.getSolution()

In [99]:
complete_image = None
for x in range(12):
    curr_col = None
    for y in range(12):
        tile_id = pos_to_id[(x, y)]
        tile_id, *tile_config = id_to_config[tile_id]
        tile = id_to_tile[tile_id].transform(tile_config).remove_borders()
        if curr_col is None:
            curr_col = tile
        else:
            curr_col = np.r_[curr_col, tile]
    
    if complete_image is None:
        complete_image = curr_col
    else:
        complete_image = np.c_[complete_image, curr_col]
        

In [100]:
masked = np.where(complete_image == ".", np.where(complete_image == "#", complete_image, 0), 1).astype(int)

In [101]:
test_image = \
""".#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###""".split("\n")
test_image = np.array(list(map(list, test_image)))
masked_test = np.where(test_image== ".", np.where(test_image == "#", test_image, 0), 1).astype(int)

In [102]:
import scipy.ndimage

dragon = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
""".split("\n")
dragon = np.array(list(map(list, dragon[:-1])))
masked_dragon = np.where(dragon == "#", dragon, 0) # If not #, becomes 0
masked_dragon = np.where(dragon == " ", masked_dragon, 1) # If not 0, becomes 1
masked_dragon = masked_dragon.astype(int)

In [104]:
for flip in [True, False]:
    for rot in [0, 1, 2, 3]:
        if flip:
            curr_image = np.fliplr(masked)
        else:
            curr_image = masked
        curr_image = np.rot90(curr_image, k=rot)
        num_dragons = (scipy.ndimage.correlate(curr_image, masked_dragon, mode="constant", cval=0) == 15).sum()
        if num_dragons > 0:
            print(masked.sum() - (num_dragons * 15))

2256
